# Enrich emails using the Sixtyfour API

In [78]:
import pandas as pd
import aiohttp
import os 
import httpx
from dotenv import load_dotenv
import asyncio
import nest_asyncio
import numpy as np
nest_asyncio.apply()
import requests

In [12]:
load_dotenv()

True

In [13]:
SIXTYFOUR_API_KEY = os.getenv('SIXTYFOUR_API_KEY')

In [67]:
df = pd.read_csv('test.csv').fillna('')

In [82]:
df.head(2)

,Unnamed: 0,name,email,phone,domain,home_address,company,previous_companies,title,linkedin,...,github_url,github_notes,experience_level,tech_stack,projects,education,Name,Profile,notes,findings
0,0,Mary Crownhart,mary@retool.com,,retool.com,,Retool,,"Lead Recruiter, GTM/G&A",https://www.linkedin.com/in/mary-crownhart-792...,...,,,,,,"University of California, Berkeley",Mary Crownhart,https://www.linkedin.com/in/mary-crownhart-792...,"Mary Crownhart is confirmed as Lead Recruiter,...","['No personal website found', 'Location: San F..."
1,1,Kimberly Wendt,,,amazon.com,,Amazon Web Services (AWS),Naval Surface Warfare Center Dahlgren Division...,Senior Solutions Architect,https://www.linkedin.com/in/kimwendt,...,,,8-10 years,,Contributions to Media & Entertainment compani...,Master's degree in Computer Science from Georg...,Kim Wendt,https://www.linkedin.com/in/kimwendt/?lipi=urn...,Kimberly Wendt is confirmed as a Senior Soluti...,"['Industry: Cloud Computing, Technology, Media..."


In [69]:
async def enrich_email(
    client: httpx.AsyncClient,
    semaphore: asyncio.Semaphore,
    lead: dict,
    max_retries: int = 3
):
    """Enrich company info using the SixtyFour API with retries and concurrency limits."""
    async with semaphore:
        for attempt in range(max_retries):
            try:
                response = await client.post(
                    "https://api.sixtyfour.ai/find-email",
                    headers={
                        'x-api-key': SIXTYFOUR_API_KEY,
                        "Content-Type": "application/json"
                    },
                    json={
                        "lead": lead
                    }
                )
                response.raise_for_status()
                print(f"Response JSON: {str(response.text)[:20]}")
                return response.json()
            except httpx.HTTPStatusError as http_err:
                print(f"HTTP error occurred: {http_err}\nResponse: {response.text}")
            except httpx.RequestError as req_err:
                print(f"Request error occurred: {req_err}")
                if attempt < max_retries - 1:
                    print(f"Retrying ({attempt + 1}/{max_retries})...")
                    await asyncio.sleep(2 ** attempt)
                    continue
            except Exception as e:
                print(f"Unexpected error: {e}")
            return None
    return None

async def process_leads(df: pd.DataFrame) -> list:
    results = []
    semaphore = asyncio.Semaphore(100)
    timeout = httpx.Timeout(300.0, connect=10.0)

    async with httpx.AsyncClient(follow_redirects=True, timeout=timeout) as client:
        tasks = [enrich_email(client, semaphore, df.iloc[i].to_dict()) for i in range(len(df))]
        results = await asyncio.gather(*tasks, return_exceptions=True)

    return results

In [70]:
results = asyncio.run(process_leads(df))

Response JSON: {"Unnamed: 0":0,"nam
Response JSON: {"Unnamed: 0":10,"na
Response JSON: {"Unnamed: 0":16,"na
Response JSON: {"Unnamed: 0":12,"na
Response JSON: {"Unnamed: 0":22,"na
Response JSON: {"Unnamed: 0":35,"na
Response JSON: {"Unnamed: 0":59,"na
Response JSON: {"Unnamed: 0":47,"na
Response JSON: {"Unnamed: 0":41,"na
Response JSON: {"Unnamed: 0":25,"na
Response JSON: {"Unnamed: 0":96,"na
Response JSON: {"Unnamed: 0":63,"na
Response JSON: {"Unnamed: 0":38,"na
Response JSON: {"Unnamed: 0":44,"na
Response JSON: {"Unnamed: 0":39,"na
Response JSON: {"Unnamed: 0":94,"na
Response JSON: {"Unnamed: 0":17,"na
Response JSON: {"Unnamed: 0":42,"na
Response JSON: {"Unnamed: 0":32,"na
Response JSON: {"Unnamed: 0":13,"na
Response JSON: {"Unnamed: 0":64,"na
Response JSON: {"Unnamed: 0":56,"na
Response JSON: {"Unnamed: 0":57,"na
Response JSON: {"Unnamed: 0":58,"na
Response JSON: {"Unnamed: 0":61,"na
Response JSON: {"Unnamed: 0":53,"na
Response JSON: {"Unnamed: 0":40,"na
Response JSON: {"Unnamed: 0"

In [71]:
final = pd.DataFrame(results)

In [73]:
final['email'] = final['email'].str[0].str[0]

In [81]:
final.fillna('').to_csv('final.csv')